# Server

In [2]:
import util

The util.py file contains a couple useful methods and packages. It contains the entire socket package, the entire Cryptodome package, as well as the following methods: 
- encrypt(message, key, nonce)
- decrypt(ciphertext, key, nonce)
- hash(message, key, nonce)
- generate_n()
- generate_k()
These methods all use various Cryptodome features, with several constants to ensure we're building based on the same model design.

In [3]:
k_ma = util.generate_k()
k_mb = util.generate_k()

server_socket = util.socket.socket(util.socket.AF_INET, util.socket.SOCK_STREAM)
server_socket.bind((util.SERVER["alias"], util.SERVER["port"]))
server_socket.listen(1)

while True:
    connection, client_address = server_socket.accept()
    try:
        data = connection.recv(1024)
        n, m_h = data[:util.SIZE_N], data[util.SIZE_N:]
        print(n)
        print(m_h)
        m, h = m_h[:-util.SIZE_K], m_h[-util.SIZE_K:]
        print(m)
        print(h)

        decrypted_data = util.decrypt(m, k_ma, n)
        if util.hash(n, decrypted_data, k_ma) == h:
            N2 = util.generate_n()
            h1 = util.hash(N2, decrypted_data, k_mb)
            m1 = util.encrypt(decrypted_data, k_mb, N2)
            connection.sendall(N2 + m1 + h1)
    finally:
        connection.close()


b'\x0b\xdb\xff)ZFT\x7f\x1cL8v\xcc\x08\\O'
b'\xc8U\x9c\xe5:\x19\xae\xe4\t\x02U\xd5\xf9\xb4B\x85F \xaa\xd7@ \x81\x1e~\x0cZC\x95\xfc)\xf3\xaa{\x08\x8c\x10\xb9\x8e\xe8Oa\xb1\xa6qh'
b'\xc8U\x9c\xe5:\x19\xae\xe4\t\x02U\xd5\xf9\xb4'
b'B\x85F \xaa\xd7@ \x81\x1e~\x0cZC\x95\xfc)\xf3\xaa{\x08\x8c\x10\xb9\x8e\xe8Oa\xb1\xa6qh'


CODE IN DEVELOPMENT:

In [1]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Hash import SHA256
import socket

from util import SERVER, generate_n, encrypt, decrypt, hash

import util

def server():
    private_key = util.SERVER_PRIVATE_KEY
    public_key = util.PUBLIC_KEY

    print("server's RSA keys:")
    #print(f"Private Key: {private_key.decode()}")
    #print(f"Public Key: {public_key.decode()}")

    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind((SERVER["alias"], SERVER["port"]))
    sock.listen(1)
    print("server listening on {}:{}".format(SERVER["alias"], SERVER["port"]))

    conn, addr = sock.accept()
    print("connected to {}".format(addr))

    received_data = conn.recv(1024)
    encrypted_nonce_p = received_data[:256]
    phone_public_key = received_data[256:]

    cipher = PKCS1_OAEP.new(RSA.import_key(private_key))
    nonce_p = cipher.decrypt(encrypted_nonce_p)

    nonce_s = generate_n()
    cipher = PKCS1_OAEP.new(RSA.import_key(public_key))
    encrypted_nonce_s = cipher.encrypt(nonce_s)
    conn.send(encrypted_nonce_s + public_key)

    encrypted_nonce_s = conn.recv(256)
    cipher = PKCS1_OAEP.new(RSA.import_key(phone_public_key))
    received_nonce_s = cipher.decrypt(encrypted_nonce_s)

    if received_nonce_s == nonce_s:
        print("3-way handshake completed")
    else:
        print("3-way handshake failed omg")

    kdc_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    kdc_sock.connect((SERVER["alias"], SERVER["port"] + 1))
    k_ms = kdc_sock.recv(util.SIZE_K)
    print(f"server received AES key (k_ms) from KDC: {k_ms.hex()}")
    
    encrypted_message = conn.recv(1024)
    cipher = PKCS1_OAEP.new(RSA.import_key(private_key))
    message = cipher.decrypt(encrypted_message)
    print(f"received message from phone: {message.decode()}")

    nonce = generate_n()
    ciphertext = encrypt(message, k_ms, nonce)
    auth_code = hash(message, k_ms, nonce)

    device_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    device_sock.connect((SERVER["alias"], SERVER["port"] + 3))
    device_sock.send(ciphertext + auth_code)

    received_data = device_sock.recv(1024)
    ciphertext = received_data[:len(received_data) - len(auth_code)]
    auth_code = received_data[len(received_data) - len(auth_code):]

    message = decrypt(ciphertext, k_ms, nonce)
    if hash(message, k_ms, nonce) == auth_code:
        print(f"received message from device: {message.decode()}")
    else:
        print("authentication failed for received message from device")

    message = b"bruh"
    cipher = PKCS1_OAEP.new(RSA.import_key(public_key))
    encrypted_message = cipher.encrypt(message)
    conn.send(encrypted_message)

    print("message sent to phone")

if __name__ == "__main__":
    server()

server's RSA keys:
server listening on localhost:1000
connected to ('127.0.0.1', 8740)
